In [2]:
import ipyparallel as ipp
import os

In [12]:
# spin up four kernals
rc = ipp.Cluster(n=4).start_and_connect_sync()

rc.wait_for_engines(n=4)
rc.ids

Starting 4 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>
100%|██████████| 4/4 [00:05<00:00,  1.40s/engine]


[0, 1, 2, 3]

In [22]:
def executeTransaction(pk):
    from web3 import Web3, AsyncWeb3
    from web3.middleware import construct_sign_and_send_raw_middleware
    import time
    import json
    import random

    # connect to the chain using the conduit RPC -- testnet so no need for access key
    w3 = Web3(Web3.HTTPProvider('https://nitrorpc-xmtp-l3-sepolia-arbitrum-anytr-x1nwrvmveu.t.conduit.xyz'))

    # Instantiate an Account object from your key:
    send_acct = w3.eth.account.from_key(pk)

    # Add acct2 as auto-signer:
    w3.middleware_onion.add(construct_sign_and_send_raw_middleware(send_acct))

    # simple function to convert recipient address to 32 byte. Note, the recipient can be anything for testing (i.e., bob) but must be <32 bytes or it will fail
    def to_bytes32(input_string):
        # Encode the string to bytes
        encoded = input_string.encode()

        # Hash the encoded bytes using KECCAK-256 (referred to as SHA3 in Ethereum)
        return Web3.keccak(encoded)

    # contract call

    # deployed contract address
    contract_address = Web3.to_checksum_address('0x7af41F8786ed496beDDEceF9341F40fEAd3F4B3F')

    # recipient address as bytes32
    msg_recipient = '0xd96FA162b064553822e765545E3D48a054eC81F6'.encode('utf-8')

    # deployed contract abi
    abi = json.loads('''[
        {
        "anonymous": false,
        "inputs": [
            {
            "indexed": true,
            "internalType": "bytes32",
            "name": "recipient",
            "type": "bytes32"
            },
            {
            "indexed": false,
            "internalType": "string",
            "name": "message",
            "type": "string"
            }
        ],
        "name": "MessageSent",
        "type": "event"
        },
        {
        "inputs": [
            {
            "internalType": "bytes32",
            "name": "recipient",
            "type": "bytes32"
            },
            {
            "internalType": "string",
            "name": "message",
            "type": "string"
            }
        ],
        "name": "send",
        "outputs": [],
        "stateMutability": "payable",
        "type": "function"
        }
    ]''')

    # Reference the deployed contract:
    send_message = w3.eth.contract(address=contract_address, abi=abi)

    # Method arguments
    # recipient's wallet address
    recipient_address = to_bytes32('0xeBFc913dFfA597C1200b4c4daDe9D90415019CC2')

    # example IPFS CID
    string_arg = 'QmPK1s3pNYLi9ERiq3BDxKa4XosgWwFRQUydHUtz4YgpqB'

    tx_count = 1
    while tx_count <= 70000:
        #if tx_count % 10 == 0:
        #print(f"executing tx {tx_count}")
        # print(f"executing tx {tx_count}")
        # print(f"nonce is {w3.eth.get_transaction_count(acct2.address)}")
        # call the send function -- note the function will be renamed to sendMessage() soon
        # Manually build and sign a transaction:
        unsent_send_message_tx = send_message.functions.send(recipient_address,string_arg).build_transaction({
            'chainId': 3105952417095944,
            "from": send_acct.address,
            "nonce": w3.eth.get_transaction_count(send_acct.address),
        })
        signed_tx = w3.eth.account.sign_transaction(unsent_send_message_tx, private_key=send_acct.key)

        # Send the raw transaction:
        tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

        # iterate
        tx_count = tx_count + 1
        
        # wait a second before sending the next tx
        random_integer = random.uniform(.99, 1.01)
        time.sleep(random_integer)

    return f"{send_acct.address} worker completed job"

In [23]:
rc[:].map_sync(executeTransaction, [os.environ.get('PRIVATE_KEY0'),os.environ.get('PRIVATE_KEY1'),
                                    os.environ.get('PRIVATE_KEY2'),os.environ.get('PRIVATE_KEY3')])